In [57]:
import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import matplotlib.pyplot as plt

torch.manual_seed(1)    # reproducible
N_Input = 100
N_Hidden = 200

In [58]:
# Data load
mat1 = sio.loadmat('Bearing/N_0.mat')
data1 = mat1['X097_DE_time']
mat2 = sio.loadmat('Bearing/IR007_0.mat')
data2 = mat2['X105_DE_time']

Normal = data1[:50000].reshape(100,int(500/N_Input),N_Input) #100 samples for normal category, each sample contains 500 points
Fault = data2[:50000].reshape(100,int(500/N_Input),N_Input) #100*5*100

In [59]:
# prepare training and test data
Normal_tensor=torch.from_numpy(Normal) # convert to torch tensor
Fault_tensor=torch.from_numpy(Fault)

#training
Train_Data=torch.cat((Normal_tensor[0:70,:,:],Fault_tensor[:70,:,:])).type(torch.FloatTensor) #put normal and fault data together (140, 10,50)
Train_Target=torch.cat((torch.zeros(70,1),torch.ones(70,1)),0).type(torch.LongTensor).squeeze() #squeeze: (140,1) ->(140,)

#test
Test_Data=torch.cat((Normal_tensor[70:,:,:],Fault_tensor[70:,:,:])).type(torch.FloatTensor)
Test_Target=torch.cat((torch.zeros(30,1),torch.ones(30,1)),0).type(torch.LongTensor).squeeze()

In [60]:
# Batch
Train_Dataset=Data.TensorDataset(Train_Data,Train_Target) #put X and Y together
Test_Dataset=Data.TensorDataset(Test_Data,Test_Target)
train_loader = Data.DataLoader(
    dataset=Train_Dataset,
    batch_size=10,
    shuffle=True) #shuffle the order of samples to train the network

In [61]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=N_Input,
            hidden_size=N_Hidden,         # rnn hidden unit
            num_layers=3,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(N_Hidden, 2)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        r_out, h_n = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(100, 200, num_layers=3, batch_first=True)
  (out): Linear(in_features=200, out_features=2, bias=True)
)


In [62]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)   # optimize all cnn parameters, sgd
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

In [ ]:
# Training and Testing
EPOCH = 50
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):   # gives batch data, normalize x when iterate train_loader
        output = rnn(b_x)            # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

    if epoch % 2 == 0:
        test_output = rnn(Test_Data) # 60*2
        pred_y = torch.max(test_output, 1)[1].data.squeeze() #60*2 ->60*1 ->(60,)
        accuracy = (pred_y == Test_Target).sum().item() / float(Test_Target.size(0))
        print('Epoch: ', epoch, '| train loss: %.4f' % loss.data, '| test accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 0.6638 | test accuracy: 0.52
Epoch:  2 | train loss: 0.0644 | test accuracy: 0.57
Epoch:  4 | train loss: 0.0479 | test accuracy: 0.57
Epoch:  6 | train loss: 0.1445 | test accuracy: 0.60
Epoch:  8 | train loss: 0.0142 | test accuracy: 0.93


In [ ]:
rnn(b_x)